In [ ]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder

In [ ]:
!gdown --id 11jHTacIQ2e3ndLseet-YQp3clAGWTpCu #ratings.csv
!gdown --id 1D1cO0G4WEUa6AAOrEHBgExjPZ5DSNn1a #credits.csv
!gdown --id 1FYYQsZYalPT3KItdbj4A29jj3setD-G9 #keywords.csv
!gdown --id 1Y7mZX40bA3dGLCsUem9Ayeooyx9TnFUp #movies-metadata.csv
!gdown --id 1mwNCe4qZoPvE_ktbHrvMNQ4hFovDsL3w #ratings_small.csv

Downloading...
From: https://drive.google.com/uc?id=11jHTacIQ2e3ndLseet-YQp3clAGWTpCu
To: /content/ratings.csv
710MB [00:07, 91.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1D1cO0G4WEUa6AAOrEHBgExjPZ5DSNn1a
To: /content/credits.csv
190MB [00:01, 96.5MB/s]


In [ ]:
df_credits = pd.read_csv("/content/credits.csv")
df_keywords = pd.read_csv("/content/keywords.csv")
df_movies = pd.read_csv("/content/movies_metadata.csv")
df_ratings = pd.concat([pd.read_csv("/content/ratings.csv"), pd.read_csv("/content/ratings_small.csv")])

# Seth - Dataframe Merge (Without Ratings Merged)

#### Removing datetime values from movie id column in df_movies

In [ ]:
index = []

for i in range(0, len(df_movies['id'])):
  data = df_movies['id'].values[i]
  if (data.find('-') != -1):
    index.append(i)

In [ ]:
print(df_movies['id'][19730])
print(df_movies['id'][29503])
print(df_movies['id'][35587])

In [ ]:
df_movies = df_movies.drop([19730, 29503, 35587])

In [ ]:
df_movies['id'] = pd.to_numeric(df_movies['id'])

#### Joining Credits and Keywords into df1

In [ ]:
df1 = df_credits.join(df_keywords.set_index('id'), on = 'id')

#### Joining Movies into df1

In [ ]:
df1 = df1.join(df_movies.set_index('id'), on = 'id')

# Merged DataFrame #1

In [ ]:
df1.head()

In [ ]:
df1.shape

In [ ]:
df1 = df1.drop_duplicates(['imdb_id'])

In [ ]:
df1.shape

# Exploratory Data Analysis (Chris)

#### Missing Data

In [ ]:
total_missing = df1.isnull().sum().sort_values(ascending=False)
percent_missing = ((df1.isnull().sum() / df1.isnull().count()) * 100).sort_values(ascending=False)
missing_values = pd.concat([total_missing, percent_missing], axis=1, keys=['Total', 'Percent'])

plt.figure(figsize=(15,10))
sns.set_style('whitegrid')
ax = sns.barplot(y=missing_values.index, x=missing_values['Total'], palette="Set2") 
ax.set_title('Total missing values by attribute')
ax.set_xlabel('Null count')
ax.set_ylabel('Attribute Name')

for p in ax.patches:
    plt.text(p.get_width()+12, p.get_y()+0.65*p.get_height(),
             '{:1.0f}'.format(p.get_width())) 

#### Movie Budget and Revenue by Year

In [ ]:
dfBudget = df1[['release_date', 'budget', 'revenue']].dropna()
dfBudget['release_date'] = pd.DatetimeIndex(dfBudget['release_date']).year
dfBudget['budget'] = dfBudget['budget'].astype(str).astype(int)
dfBudget['budget'] = dfBudget['budget'].apply(lambda x: round(x/1000000))
dfBudget['revenue'] = dfBudget['revenue'].apply(lambda x: round(x/1000000))


plt.figure(figsize=(20,10))
plt.subplot(1, 2, 1)
plt.scatter(x = 'release_date' ,y = 'budget' , data = dfBudget)
plt.xlabel("Year")
plt.ylabel("Budget (Millions)")
plt.title("Budget by Year")

plt.subplot(1, 2, 2)
plt.scatter(x = 'release_date' ,y = 'revenue' , data = dfBudget)
plt.xlabel("Year")
plt.ylabel("Revenue (Millions)")
plt.title("Revenue by Year")
plt.show()

#### AverageBudget/Revenue by Month

In [ ]:
dfBudget = df1[['release_date', 'budget', 'revenue']].dropna()
dfBudget['release_date'] = pd.DatetimeIndex(dfBudget['release_date']).month
dfBudget['budget'] = dfBudget['budget'].astype(str).astype(int)
dfBudget['budget'] = dfBudget['budget'].apply(lambda x: round(x/1000000))
dfBudget['revenue'] = dfBudget['revenue'].apply(lambda x: round(x/1000000))
dfBudget = dfBudget.groupby('release_date')['budget', 'revenue'].mean()

plt.figure(figsize=(20,10))
plt.bar(dfBudget.index , dfBudget['budget'], label='budget', color= 'r')
plt.bar(dfBudget.index , dfBudget['revenue'], label='revenue', alpha = 0.4, color= 'b')
plt.xlabel("Month")
plt.ylabel("Budget / Revenue (Millions)")
plt.title("Budget/Revenue by Month")
plt.legend(loc='best')
plt.show()


#### Top 20 Movies Profitability

In [ ]:
dfProfit = df1[['original_title', 'budget', 'revenue']].dropna()
dfProfit['budget'] = dfProfit['budget'].astype(str).astype(int)
dfProfit['revenue'] = dfProfit['revenue'].astype(int)
#Remove rows where the budget or revenue is zero because they would not provide a fair comparison
dfProfit = dfProfit[dfProfit.revenue != 0]
dfProfit = dfProfit[dfProfit.budget != 0]
dfProfit['profit'] = dfProfit.apply(lambda row: int(row['revenue']) - row['budget'], axis=1)

plt.figure(figsize=(20,10))
sns.set_style('whitegrid')
plt.subplot(1, 2, 1)
ax = sns.barplot(y='original_title', x='profit', data=dfProfit.sort_values('profit', ascending=False)[:20], palette="cubehelix") 
ax.set_title('Top 20 Movies (Best profitability)')
ax.set_xlabel("Profit (Billion)")
ax.set_ylabel('Name')

plt.subplot(1, 2, 2)
ax = sns.barplot(y='original_title', x='profit', data=dfProfit.sort_values('profit')[:20], palette="cubehelix") 
ax.set_title('Top 20 Movies (Worst profitability)')
ax.set_xlabel("Profit (Billion)")
ax.set_ylabel('Name')
plt.subplots_adjust(wspace=0.7)
plt.show()

#### Movies Per Year

In [ ]:
dfYear = df1[['release_date']].dropna()
dfYear['release_date'] = pd.DatetimeIndex(dfYear['release_date']).year
dfdfBudget = dfBudget.groupby('release_date')['budget', 'revenue'].mean()
dfYear = dfYear.groupby('release_date')['release_date'].count().reset_index(name="count")

plt.figure(figsize=(20,10))
sns.set_style('whitegrid')
ax = sns.lineplot(x='release_date', y='count', data=dfYear) 
ax.set_title('Movies Per Year')
ax.set_xlabel("Year")
ax.set_ylabel('Number of Movies')

#### Movies by Genre

In [ ]:
# Clean genres column
def clean_genre(row):
  genres = []
  gen_dict = eval(row)
  for i in range(len(gen_dict)):
    genres.append(gen_dict[i]['name'])

  return genres

In [ ]:
df1['genres'] = df1['genres'].apply(lambda x: clean_genre(x))
dfGenres = df1[['genres']].dropna()
dfGenres = dfGenres[dfGenres['genres'].map(lambda x: len(x)) > 0]

dfGenres= pd.Series([x for item in dfGenres.genres for x in item]).value_counts().reset_index()
dfGenres.columns = ['genre', 'count']

plt.figure(figsize=(20,10))
sns.set_style('whitegrid')
ax = sns.barplot(y='genre', x='count', data=dfGenres.sort_values('count', ascending=False), palette="flare") 
ax.set_title('Movies by Genre')
ax.set_xlabel("Number of Movies")
ax.set_ylabel('Genre')

for p in ax.patches:
    plt.text(p.get_width()+12, p.get_y()+0.65*p.get_height(),
             '{:1.0f}'.format(p.get_width())) 

#### Top 20 Actors

In [ ]:
# Clean cast column
def clean_cast(row):
  cast = []
  cast_dict = eval(row)
  for i in range(len(cast_dict)):
    cast.append(cast_dict[i]['name'])

  return cast

In [ ]:
df1['cast'] = df1['cast'].apply(lambda x: clean_cast(x))
dfCast = df1[['cast']].dropna()
dfCast = dfCast[dfCast['cast'].map(lambda x: len(x)) > 0]

dfCast= pd.Series([x for item in dfCast.cast for x in item]).value_counts().reset_index()
dfCast.columns = ['actor', 'count']

plt.figure(figsize=(20,10))
sns.set_style('whitegrid')
ax = sns.barplot(y='actor', x='count', data=dfCast.sort_values('count', ascending=False)[:20], palette="crest") 
ax.set_title('Top 20 Actors')
ax.set_xlabel("Number of Movies")
ax.set_ylabel('Actor')

for p in ax.patches:
    plt.text(p.get_width()+1, p.get_y()+0.65*p.get_height(),
             '{:1.0f}'.format(p.get_width())) 

#### Movie Rating vs Popularity

In [ ]:
dfPopRating = df1[['popularity', 'vote_average']].dropna()
dfPopRating['popularity'] = dfPopRating['popularity'].astype(float)
plt.figure(figsize=(20,10))
ax = sns.scatterplot(x = 'popularity' ,y = 'vote_average' , data = dfPopRating)
ax.set_title('Rating vs Popularity')
ax.set_xlabel("Popularity")
ax.set_ylabel("Rating")

#### Overview Word Cloud

In [ ]:
dfOverView = df1[['overview']]
text = ' '.join(dfOverView['overview'].fillna('').values)
wordcloud = WordCloud(margin=10, background_color='white', colormap='Blues', width=1200, height=1000).generate(text)
plt.figure(figsize = (10, 10))
plt.imshow(wordcloud, interpolation='bilinear')
plt.title('Top words in overview', fontsize=20)
plt.axis('off')
plt.show()

# Cleaning DF1

In [ ]:
del df1['crew']
del df1['homepage']
del df1['poster_path']
del df1['imdb_id']
del df1['overview']
del df1['tagline']
del df1['budget']
del df1['status']
del df1['video']
del df1['spoken_languages']
del df1['production_countries']
del df1['runtime']
del df1['original_title']
del df1['belongs_to_collection']
del df1['production_companies']

In [ ]:
df1.head()

In [ ]:
df1.dtypes

In [ ]:
# Clean keywords column
def clean_keywords(row):
  keywords = []
  keyword_dict = eval(row)
  for i in range(len(keyword_dict)):
    keywords.append(keyword_dict[i]['name'])

  return keywords

In [ ]:
df1['keywords'] = df1['keywords'].apply(lambda x: clean_keywords(x))

In [ ]:
df1.head()

### Creating a Cast DF

In [ ]:
cast = eval (df_credits['cast'].values[0])

In [ ]:
cast[0]['id']

In [ ]:
ids = []
actors = []

for i in range(0, len(df_credits['cast'])):
  castData = eval(df_credits['cast'].values[i])
  for j in range(len(castData)):
    ids.append(castData[j]['id'])
    actors.append(castData[j]['name'])

In [ ]:
castDF = pd.DataFrame({'id': ids, 'name': actors})

In [ ]:
castDF.head()

In [ ]:
castDF[castDF.duplicated()]

In [ ]:
castDF.shape

Remove Duplicates

In [ ]:
castDF.drop_duplicates(keep = 'first', inplace = True)

In [ ]:
castDF.shape

## Finding unique cast number for each movie

In [ ]:
# convert castDF into dictionary
castDict = dict(zip(castDF.name, castDF.id))

In [ ]:
castDict

In [ ]:
def get_cast_number(row):
  if len(row) == 0:
    return 0
  else:
    array = np.vectorize(castDict.get)(row)
    return sum(array)

In [ ]:
df1['castNum'] = df1['cast'].apply(lambda x: get_cast_number(x))

In [ ]:
df1.head()

## Find Genre Number for Each Movie

In [ ]:
def convert_genres(row):
  sum = 0
  for i in row:
    for j in i:
      sum+=ord(j)
  return sum

In [ ]:
df1['genreNum'] = df1['genres'].apply(lambda x: convert_genres(x))

In [ ]:
df1.head()

## Convert Title to Number

In [ ]:
df1[df1['title'].apply(lambda x: isinstance(x, float))]

Need to remove these columns since they have NaN values

In [ ]:
df1 = df1.drop([19729, 29515, 35598, 35598])

In [ ]:
def convert_titles(row):
  sum = 1;
  for i in row:
    sum+=ord(i)
  
  return(sum)

In [ ]:
# def convert_titles2(row):
#   return int.from_bytes(row.encode(), 'little')

In [ ]:
df1['titleNum'] = df1['title'].apply(lambda x: convert_titles(x))

In [ ]:
df1.head()

In [ ]:
# copy
df_final = df1.copy()

In [ ]:
# df1 = df_final

## Chris - Clean & Transform the keywords column

#### Convert the array to string

In [ ]:
df1['keywords'] = df1['keywords'].apply(', '.join)
df1.head()

#### Convert to lowercast

In [ ]:
df1['keywords'] = df1['keywords'].str.lower()

#### Remove punctuations

In [ ]:
df1['keywords'] = df1['keywords'].str.replace('[^\w\s]','')

###### Remove links

In [ ]:
df1['keywords'] = df1['keywords'].str.replace('http\S+|www.\S+', '', case=False)

###### Remove html tags

In [ ]:
def rmtags(keyword):
  rmtag = re.compile('<.*?>')
  keywordtext = re.sub(rmtag, '', keyword)
  return keywordtext

df1['keywords'] = df1['keywords'].apply(lambda keyword: rmtags(keyword))

##### Remove additional whitespace

In [ ]:
df1['keywords'] = df1['keywords'].replace(' +', ' ', regex=True)
print(df1['keywords'][0])
print('\n----------\n')
print(df1['keywords'][1])

### Encode the keywords with TFIDFVectorizer

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words = 'english', ngram_range=(1,1))
tfidf = tfidf_vectorizer.fit_transform(df1['keywords'])

In [ ]:
keywordNum = np.sum(tfidf.A, axis=1)

In [ ]:
df1['keywords'] = keywordNum

In [ ]:
df1.head()

In [ ]:
title_df = df1['title']

In [ ]:
del df1['cast']
del df1['genres']
del df1['title']

In [ ]:
df1.head()

### Convert remaining columns to numeric

In [ ]:
# change popularity to float
df1['popularity'] = df1['popularity'].apply(pd.to_numeric)

In [ ]:
# convert adult true or false to 1 or 0
adult_encoder = LabelEncoder()
df1["adult"] = adult_encoder.fit_transform(df1["adult"])

In [ ]:
# change date to date-time
df1['release_date'] = pd.to_datetime(df1['release_date'])

In [ ]:
categorical = df1.dtypes[df1.dtypes == "object"].index
categorical

In [ ]:
for c in categorical:
    df1[c] = df1[c].astype('category')
    df1[c] = df1[c].cat.codes

In [ ]:
df1.dtypes

In [ ]:
df2 = df1[['id', 'keywords', 'adult', 'original_language', 'popularity', 'revenue', 'vote_average', 'vote_count', 'castNum', 'genreNum', 'titleNum']]
df2['title'] = title_df.str.lower()

In [ ]:
df2.head(10)

In [ ]:
df2.shape

In [ ]:
df2 = df2.drop_duplicates(['id'])
df2.head(10)

In [ ]:
df2.shape

(45429, 12)

### Convert the Cleaned and Tranformed Dataframe to csv for use in the recommender system & Streamlit

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
df2.to_csv('/content/drive/My Drive/Big Data Project/data/KaggleMovies.csv', index=False)